In [4]:
import numpy as np
import pandas as pd
import optuna
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_percentage_error

# Step 1: Load the data
tesla_data = pd.read_csv('Clean_data/Cleaned_Tesla_Close.csv', parse_dates=['Date'], index_col='Date')
sp500_data = pd.read_csv('Clean_data/Cleaned_SP500_Close.csv', parse_dates=['Date'], index_col='Date')
ixic_data = pd.read_csv('Clean_data/Cleaned_IXIC_Close.csv', parse_dates=['Date'], index_col='Date')

# Merge datasets on 'Date'
merged_data = tesla_data.merge(sp500_data, left_index=True, right_index=True, suffixes=('', '_sp500'))
merged_data = merged_data.merge(ixic_data, left_index=True, right_index=True, suffixes=('', '_ixic'))

# Step 2: Preprocess the data
data = merged_data[['Close_predicted', 'Close_predicted_sp500', 'Close_predicted_ixic']]

# Scale the data after making it stationary
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data.dropna())  # Ensure NaN values are removed after differencing
# Prepare the dataset for LSTM
def create_dataset(dataset, time_step=60):
    X, y = [], []
    for i in range(time_step, len(dataset)):
        X.append(dataset[i - time_step:i])
        y.append(dataset[i, 0])  # Predict the 'Close' price of Tesla
    return np.array(X), np.array(y)

time_step = 60
X, y = create_dataset(scaled_data, time_step)

# Split data into training and test sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]


# Function to create and train the LSTM model with Optuna
def objective(trial):
    # Hyperparameters to optimize
    lstm_units = trial.suggest_int('lstm_units', 78, 83)
    dropout_rate = trial.suggest_float('dropout_rate', 0.43, 0.48)
    learning_rate = trial.suggest_float('learning_rate', 0.004, 0.005, log=True)
    batch_size = trial.suggest_categorical('batch_size', [16, 32])
    epochs = trial.suggest_int('epochs', 100, 200)

    # Build the LSTM model
    model = Sequential([
        Input(shape=(time_step, X_train.shape[2])),
        LSTM(lstm_units, return_sequences=True),
        Dropout(dropout_rate),
        LSTM(lstm_units),
        Dropout(dropout_rate),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error')

    # Train the model
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)

    # Evaluate the model
    test_predictions = model.predict(X_test)
    test_predictions_rescaled = scaler.inverse_transform(
        np.hstack([test_predictions, X_test[:, -1, 1:]])
    )[:, 0]
    y_test_rescaled = scaler.inverse_transform(
        np.hstack([y_test.reshape(-1, 1), X_test[:, -1, 1:]])
    )[:, 0]

    # Calculate MAPE
    mape = np.mean(np.abs((y_test_rescaled - test_predictions_rescaled) / y_test_rescaled)) * 100
    return mape

# Run the optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

# Print the best hyperparameters
print('Best trial:')
trial = study.best_trial
print(f'  Value (MAPE): {trial.value}')
print('  Params:')
for key, value in trial.params.items():
    print(f'    {key}: {value}')


[I 2024-11-16 23:45:39,050] A new study created in memory with name: no-name-db9aa1a1-311e-42ac-a63f-c068b7b5b60e


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


[I 2024-11-16 23:47:22,342] Trial 0 finished with value: 7.2338199740118085 and parameters: {'lstm_units': 78, 'dropout_rate': 0.4706038022776564, 'learning_rate': 0.0042110915714095535, 'batch_size': 32, 'epochs': 125}. Best is trial 0 with value: 7.2338199740118085.


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step


[I 2024-11-16 23:49:02,192] Trial 1 finished with value: 5.971556145582714 and parameters: {'lstm_units': 80, 'dropout_rate': 0.45638738927277317, 'learning_rate': 0.0040302777941638454, 'batch_size': 32, 'epochs': 124}. Best is trial 1 with value: 5.971556145582714.


...

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


[I 2024-11-17 03:51:32,515] Trial 98 finished with value: 9.365261698267012 and parameters: {'lstm_units': 78, 'dropout_rate': 0.46328505331479664, 'learning_rate': 0.0047013893176539616, 'batch_size': 32, 'epochs': 170}. Best is trial 64 with value: 5.153969000460726.


9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step


[I 2024-11-17 03:54:07,075] Trial 99 finished with value: 6.688106810581412 and parameters: {'lstm_units': 83, 'dropout_rate': 0.46894090699743796, 'learning_rate': 0.004267001183481356, 'batch_size': 32, 'epochs': 190}. Best is trial 64 with value: 5.153969000460726.


Best trial:
  Value (MAPE): 5.153969000460726
  Params:
    lstm_units: 78
    dropout_rate: 0.4375723794875016
    learning_rate: 0.00422063298445162
    batch_size: 32
    epochs: 176
